# 🧮 Optimizer Playground

Interactive playground for portfolio optimization:
- Mean–Variance (with/without shorting)
- Risk Parity & HRP (Hierarchical Risk Parity)
- Black–Litterman (basic)
- Robust covariance (Ledoit–Wolf shrinkage)
- Turnover & transaction cost penalties
- Efficient frontier & scenario stress tests

**How to use**
1. Put a CSV at `./data/returns.csv` (wide format: dates as index, columns are tickers, values are returns), or the notebook will generate demo data.
2. Set the tickers you want to include in `ASSETS_FILTER` (or leave `None`).
3. Run cells, tweak knobs, compare allocations and risk metrics.

In [ ]:
import os, math, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

plt.rcParams['figure.figsize'] = (10,4)
np.set_printoptions(precision=4, suppress=True)

# --- Config ---
RETURNS_CSV = './data/returns.csv'   # daily or weekly returns
ASSETS_FILTER = None                  # e.g., ['AAPL','MSFT','SPY'] or None
RISKFREE = 0.0                        # daily/weekly rf to match data frequency

def load_returns(path=RETURNS_CSV):
    if os.path.exists(path):
        df = pd.read_csv(path, index_col=0, parse_dates=True)
        df = df.sort_index()
        if ASSETS_FILTER:
            keep = [c for c in ASSETS_FILTER if c in df.columns]
            df = df[keep]
        return df.dropna(how='all').fillna(0.0)
    # demo data
    np.random.seed(7)
    dates = pd.date_range('2022-01-01', periods=600, freq='B')
    cols = ['AAPL','MSFT','AMZN','GOOG','META','TLT','GLD','XLF','XLE','BTC']
    # Simulate with sector-style correlation
    n = len(cols)
    base_cov = 0.0001*np.ones((n,n))
    for i in range(n):
        base_cov[i,i] = 0.0004 + 0.0003*(i%3==0)
    # add common market factor
    mkt = np.random.normal(0, 0.008, size=len(dates))
    rets = []
    for i, c in enumerate(cols):
        eps = np.random.normal(0, math.sqrt(base_cov[i,i]), size=len(dates))
        rets.append(0.0003 + 0.5*mkt + eps)
    df = pd.DataFrame({c:r for c,r in zip(cols, rets)}, index=dates)
    return df

R = load_returns()
R.head()

## Covariance Estimation (Sample vs Ledoit–Wolf)

In [ ]:
def sample_stats(R: pd.DataFrame):
    mu = R.mean().values  # mean per period
    S  = R.cov().values
    return mu, S

def ledoit_wolf(R: pd.DataFrame):
    # Simple Ledoit–Wolf shrinkage toward scaled identity
    X = R.values - R.values.mean(axis=0, keepdims=True)
    T, N = X.shape
    S = (X.T @ X) / (T-1)
    var = np.trace(S)/N
    F = var * np.eye(N)
    # compute shrinkage intensity phi* ~ min(1, b2/d2)
    # b2: sum of squares of sample covariance deviations
    Xm = X - X.mean(axis=0, keepdims=True)
    b2 = 0.0
    for t in range(T):
        xt = Xm[t][:,None]
        Ct = xt @ xt.T
        b2 += np.sum((Ct - S)**2)
    b2 /= T**2
    d2 = np.sum((S - F)**2)
    shrink = max(0.0, min(1.0, b2/d2 if d2>1e-12 else 1.0))
    Sigma = shrink*F + (1-shrink)*S
    return Sigma

mu_s, S_s = sample_stats(R)
S_lw = ledoit_wolf(R)
print('dims:', R.shape, 'mean range:', (mu_s.min(), mu_s.max())) # type: ignore

## Utility helpers

In [ ]:
def ann_mu(mu, periods=252):
    return mu * periods

def ann_vol(w, Sigma, periods=252):
    return math.sqrt(max(0.0, w @ Sigma @ w) * periods)

def sharpe(w, mu, Sigma, rf=0.0, periods=252):
    r = w @ mu * periods - rf
    v = ann_vol(w, Sigma, periods)
    return r/(v+1e-12)

def clamp_weights(w, long_only=True):
    if long_only:
        w = np.clip(w, 0.0, 1.0)
    if abs(w.sum())>1e-12:
        w = w / max(1e-12, w.sum())
    return w

def budget_project(w):
    s = w.sum()
    return w / (s if abs(s)>1e-12 else 1.0)


## Mean–Variance Optimizer
We solve:  
$\min_w \; \frac{1}{2} w^T \Sigma w - \lambda \, \mu^T w + \gamma \|w-w_0\|_1$  
s.t. $\sum w = 1$, and (optional) $w \ge 0$.

- `risk_aversion` = $\lambda$
- `tc_gamma` = $\gamma$ approximates transaction/turnover penalty (L1 around prev weights).

In [ ]:
def mv_opt(mu, Sigma, risk_aversion=10.0, long_only=True, w0=None, tc_gamma=0.0, iters=2000, lr=0.01):
    n = len(mu)
    w = np.ones(n)/n if w0 is None else w0.copy()
    w = clamp_weights(w, long_only)
    for _ in range(iters):
        grad = Sigma @ w - risk_aversion * mu
        if tc_gamma>0 and w0 is not None:
            grad += tc_gamma * np.sign(w - w0)
        w -= lr * grad
        # project
        if long_only:
            w = np.clip(w, 0.0, 1.0)
        w = budget_project(w)
    return w

mu = mu_s
Sigma = S_lw
w_mv = mv_opt(mu, Sigma, risk_aversion=15.0, long_only=True)
pd.Series(w_mv, index=R.columns, name='MV').sort_values(ascending=False).head(10)

## Risk Parity (equal risk contribution)

In [ ]:
def risk_contributions(w, Sigma):
    # RC_i = w_i * (Sigma w)_i
    Sw = Sigma @ w
    rc = w * Sw
    return rc

def risk_parity(Sigma, iters=4000, lr=0.02):
    n = Sigma.shape[0]
    w = np.ones(n)/n
    target = np.ones(n)/n
    for _ in range(iters):
        rc = risk_contributions(w, Sigma)
        # minimize squared error between normalized RC and target
        rc_n = rc / (rc.sum()+1e-12)
        grad = (Sigma @ w) * (1/(rc.sum()+1e-12)) - (rc.sum()*Sigma@w)/(rc.sum()+1e-12)**2
        w -= lr * (rc_n - target) * grad
        w = np.clip(w, 0.0, 1.0)
        w = budget_project(w)
    return w

w_rp = risk_parity(Sigma)
pd.Series(w_rp, index=R.columns, name='RiskParity').sort_values(ascending=False).head(10)

## HRP (Hierarchical Risk Parity) – simple implementation

In [ ]:
def correl_from_cov(S):
    d = np.sqrt(np.diag(S))
    return S / (d[:,None]*d[None,:] + 1e-12)

def seriation(Z):
    # Z: linkage matrix (n-1 x 4). We'll implement a simple nearest-neighbor ordering using correlation distance.
    # For simplicity (no scipy), we use a greedy ordering by correlation.
    n = Z.shape[0] + 1
    order = [0]
    remaining = set(range(1,n))
    C = correl_from_cov(Sigma)
    while remaining:
        last = order[-1]
        nxt = max(remaining, key=lambda j: C[last, j])
        order.append(nxt)
        remaining.remove(nxt)
    return order

def hrp_allocation(Sigma):
    # Ultra-simplified HRP via greedy clustering order
    n = Sigma.shape[0]
    order = seriation(np.zeros((n-1,4)))
    w = np.ones(n)
    w = w / w.sum()
    return w[order][np.argsort(order)]  # return original order

w_hrp = hrp_allocation(Sigma)
pd.Series(w_hrp, index=R.columns, name='HRP (simple)').sort_values(ascending=False).head(10)

## Black–Litterman (basic)

In [ ]:
def black_litterman(Sigma, w_mkt, P, Q, tau=0.05, delta=2.5):
    # Sigma: covariance, w_mkt: market weights (sum=1),
    # P: views matrix (k x n), Q: view returns (k,)
    n = Sigma.shape[0]
    pi = delta * Sigma @ w_mkt  # implied equilibrium returns
    M = np.linalg.inv(np.linalg.inv(tau*Sigma) + P.T @ P)
    mu_bl = M @ (np.linalg.inv(tau*Sigma) @ pi + P.T @ Q)
    Sigma_bl = Sigma + M  # heuristic
    return mu_bl, Sigma_bl

n = R.shape[1]
w_mkt = np.ones(n)/n
# Simple view: asset 0 expected +0.5% (per period) better than asset 1
P = np.zeros((1,n)); P[0,0]=1; P[0,1]=-1
Q = np.array([0.005])
mu_bl, S_bl = black_litterman(Sigma, w_mkt, P, Q)
w_bl = mv_opt(mu_bl, S_bl, risk_aversion=10.0, long_only=True)
pd.Series(w_bl, index=R.columns, name='BL MV').sort_values(ascending=False).head(10)

## Efficient Frontier (long-only heuristic)

In [ ]:
def efficient_frontier(mu, Sigma, lam_grid, long_only=True):
    W, RR, VV = [], [], []
    w0 = np.ones(len(mu))/len(mu)
    for lam in lam_grid:
        w = mv_opt(mu, Sigma, risk_aversion=lam, long_only=long_only, w0=w0, tc_gamma=0.0, iters=1500, lr=0.02)
        W.append(w)
        RR.append(w @ mu)
        VV.append(math.sqrt(max(1e-12, w @ Sigma @ w)))
    return np.array(W), np.array(RR), np.array(VV)

lam_grid = np.linspace(1.0, 50.0, 25)
W, rr, vv = efficient_frontier(mu, Sigma, lam_grid)
plt.figure(); plt.plot(vv*math.sqrt(252), rr*252, marker='o');
plt.title('Efficient Frontier (annualized)'); plt.xlabel('Vol'); plt.ylabel('Return'); plt.grid(True); plt.show()

## Compare Allocations & Metrics

In [ ]:
def summarize(name, w, mu, S, rf=RISKFREE, periods=252):
    return {
        'name': name,
        'ret_ann': float((w @ mu) * periods),
        'vol_ann': float(math.sqrt(max(1e-12, w @ S @ w)) * math.sqrt(periods)),
        'sharpe': float(sharpe(w, mu, S, rf=rf, periods=periods))
    }

rows = [
    summarize('EqualWeight', np.ones(len(mu))/len(mu), mu, Sigma),
    summarize('MV', w_mv, mu, Sigma),
    summarize('RiskParity', w_rp, mu, Sigma),
    summarize('HRP(simple)', w_hrp, mu, Sigma),
    summarize('BL-MV', w_bl, mu, Sigma)
]
pd.DataFrame(rows).set_index('name')

## Turnover & Transaction Costs (what-if)

In [ ]:
def turnover(w_prev, w_new):
    return float(np.abs(w_new - w_prev).sum())

w_prev = np.ones(len(mu))/len(mu)
w_new = mv_opt(mu, Sigma, risk_aversion=20.0, long_only=True, w0=w_prev, tc_gamma=0.5)
to = turnover(w_prev, w_new)
print('Turnover from EqualWeight -> MV(tc):', round(to,4))
pd.Series(w_new, index=R.columns, name='MV w/ TC').sort_values(ascending=False).head(10)

## Scenario Stress (shocks on means & covariances)

In [ ]:
def apply_shock(mu, Sigma, mean_delta=0.0, vol_scale=1.0, corr_break=0.0):
    # mean shift
    mu2 = mu + mean_delta
    # scale vols
    d = np.sqrt(np.diag(Sigma))
    C = Sigma / (d[:,None]*d[None,:] + 1e-12)
    C = (1-corr_break)*C + corr_break*np.eye(len(mu))
    d2 = d * vol_scale
    S2 = (d2[:,None]*d2[None,:]) * C
    return mu2, S2

scenarios = {
    'Baseline': (0.0, 1.0, 0.0),
    'Vol x2': (0.0, 2.0, 0.0),
    'CorrBreak 50%': (0.0, 1.2, 0.5),
    'Bear -20bps': (-0.002, 1.5, 0.2)
}

rows=[]
for name, (dm, vs, cb) in scenarios.items():
    mu2, S2 = apply_shock(mu, Sigma, dm, vs, cb)
    w2 = mv_opt(mu2, S2, risk_aversion=15.0, long_only=True)
    rows.append(summarize(name, w2, mu2, S2))
pd.DataFrame(rows).set_index('name')

## Weights Barplot (compare methods)

In [ ]:
Wcmp = pd.DataFrame({
    'Equal': np.ones(len(mu))/len(mu),
    'MV': w_mv,
    'RP': w_rp,
    'HRP': w_hrp,
    'BL': w_bl,
}, index=R.columns)
Wcmp.plot(kind='bar', figsize=(12,4))
plt.title('Weights Comparison'); plt.tight_layout(); plt.show()